In [4]:
import torch 
from torch.utils import data
from torch import nn

from typing import List, Tuple, Dict, Union, Optional
from typing import Iterable

import pandas as pd
import numpy as np
import surprise

from tqdm.notebook import tqdm

In [5]:
import sys
import os 

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

sys.path.insert(0, r'./modules/')

In [6]:
from modules import AutoRec, Model, CollobarativeModel
from modules import get_validation_data, Dataset
from modules import nmf, svd, knn, autorec

In [7]:
to_dataset = lambda x: Dataset(x, 
                  validation_data.user_id.unique().shape[0], 
                  validation_data.item_id.unique().shape[0], 
                  user_based=True,
                  long_matrix=True,
                  user_column="user_id", item_column="item_id", rating_column="rating")

In [8]:
validation_data = pd.read_csv(os.path.join('../..', r"./synthetic_data/validation_data/dense_"+str(13)+'.csv'))
heristic = pd.read_csv(os.path.join('../..', r"./synthetic_data/validation_data/dense_hue_"+str(13)+'.csv'))

## Sparse case

In [7]:
SAMPLE_SIZE = [0.1, 0.5, 0.9]
STEPS = 20
SEED = 2022

In [8]:
result = pd.DataFrame()

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

np.random.seed(SEED)

In [ ]:
SAMPLE_SIZE = [0.1, 0.5, 0.9]
for sample in tqdm(SAMPLE_SIZE):
    data_sparse = validation_data.sample(frac=0.05)
    data_sparse = data_sparse.sample(frac=sample)

    data_train = data_sparse.sample(frac=0.8)
    data_test = data_sparse.drop(data_train.index)
    heu_train = heristic.loc[data_train.index]
    heu_test = heristic.loc[data_test.index]

    data_train = to_dataset(data_train)
    data_test = to_dataset(data_test)
    heu_train = to_dataset(heu_train)
    heu_test = to_dataset(heu_test)
    random_train = torch.randint(low=0, high=11, size=data_train[:].shape)*data_train[:].sign()
    random_test = torch.randint(low=0, high=11, size=data_test[:].shape)*data_test[:].sign()

    for i in range(0, STEPS+1):
        for j in range(0, STEPS-i+1):
            k = STEPS - i - j
            a1, a2, a3 = i/STEPS, j/STEPS, k/STEPS
            
            print()
            print(a1, a2, a3)
            print("----------------------------------------------")
            print()
            
            mtp_train = a1*data_train[:]+a2*heu_train[:]+a3*random_train[:]
            mtp_test = a1*data_test[:]+a2*heu_test[:]+a3*random_test[:]

            mtp_train = Dataset(mtp_train, 
                                validation_data.user_id.unique().shape[0], 
                                validation_data.item_id.unique().shape[0], 
                                user_based=True,
                                long_matrix=False,
                                user_column="user_id", item_column="item_id", rating_column="rating")

            mtp_test = Dataset(mtp_test, 
                               validation_data.user_id.unique().shape[0], 
                               validation_data.item_id.unique().shape[0], 
                               user_based=True,
                               long_matrix=False,
                               user_column="user_id", item_column="item_id", rating_column="rating")
            
            svd_result = svd([mtp_train], [mtp_test], sample, cnt=1, with_null=False)
            svd_result['a1']=a1
            svd_result['a2']=a2
            svd_result['a3']=a3
            
            nmf_result = nmf([mtp_train], [mtp_test], sample, cnt=1, with_null=False)
            nmf_result['a1']=a1
            nmf_result['a2']=a2
            nmf_result['a3']=a3
            
            knn_result = knn([mtp_train], [mtp_test], sample, cnt=1, with_null=False)
            knn_result['a1']=a1
            knn_result['a2']=a2
            knn_result['a3']=a3
            
            autorec_result = autorec([mtp_train], [mtp_test], validation_data, sample, cnt=1, with_null=True)
            autorec_result['a1']=a1
            autorec_result['a2']=a2
            autorec_result['a3']=a3
            
            result = result.append(svd_result)
            result = result.append(nmf_result)
            result = result.append(knn_result)
            result = result.append(autorec_result)

  0%|          | 0/3 [00:00<?, ?it/s]


0.0 0.0 1.0
----------------------------------------------

RMSE: 2.9625
MAE:  2.6836
RMSE: 3.1565
MAE:  2.8366
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.9143
MAE:  2.6167

0.0 0.05 0.95
----------------------------------------------

RMSE: 3.0833
MAE:  2.8465
RMSE: 3.3633
MAE:  3.1257
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 3.0494
MAE:  2.7920

0.0 0.1 0.9
----------------------------------------------

RMSE: 2.9528
MAE:  2.7222
RMSE: 3.2863
MAE:  3.0738
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.9227
MAE:  2.6708

0.0 0.15 0.85
----------------------------------------------

RMSE: 2.8312
MAE:  2.6113
RMSE: 3.1434
MAE:  2.9306
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.7993
MAE:  2.5496

0.0 0.2 0.8
----------------------------------------------

RMSE: 2.7006
MAE:  2.4680
RMSE: 3.0667
MAE:  2.8538
Computing the msd similarity matrix

/data/home/agurov/DeepRecSys/models.py:176: RuntimeWarning: Mean of empty slice
  result = {"rmse":np.sqrt(rmse/cnt), "mae": mae/cnt, "ndcg":np.nanmean(ndcg)}



0.0 0.45 0.55
----------------------------------------------

RMSE: 2.1535
MAE:  1.8750
RMSE: 2.5503
MAE:  2.2656
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.1611
MAE:  1.8688


/data/home/agurov/DeepRecSys/models.py:176: RuntimeWarning: Mean of empty slice
  result = {"rmse":np.sqrt(rmse/cnt), "mae": mae/cnt, "ndcg":np.nanmean(ndcg)}


...

/data/home/agurov/DeepRecSys/models.py:176: RuntimeWarning: Mean of empty slice
  result = {"rmse":np.sqrt(rmse/cnt), "mae": mae/cnt, "ndcg":np.nanmean(ndcg)}



0.1 0.3 0.6
----------------------------------------------

RMSE: 2.1557
MAE:  1.9334
RMSE: 2.5100
MAE:  2.2775
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.1576
MAE:  1.9117

0.1 0.35 0.55
----------------------------------------------

RMSE: 2.0554
MAE:  1.8228
RMSE: 2.4546
MAE:  2.2075
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 2.0589
MAE:  1.8073

0.1 0.4 0.5
----------------------------------------------

RMSE: 1.9565
MAE:  1.7086
RMSE: 2.4350
MAE:  2.1564
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.9683
MAE:  1.7115

0.1 0.45 0.45
----------------------------------------------

RMSE: 1.8687
MAE:  1.6055
RMSE: 2.2839
MAE:  2.0063
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.8868
MAE:  1.6159

0.1 0.5 0.4
----------------------------------------------

RMSE: 1.7906
MAE:  1.5171
RMSE: 2.1912
MAE:  1.9064
Computing the msd similarity matrix

/data/home/agurov/DeepRecSys/models.py:176: RuntimeWarning: Mean of empty slice
  result = {"rmse":np.sqrt(rmse/cnt), "mae": mae/cnt, "ndcg":np.nanmean(ndcg)}



0.1 0.55 0.35
----------------------------------------------

RMSE: 1.7197
MAE:  1.4426
RMSE: 2.1579
MAE:  1.8617
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.7563
MAE:  1.4737

0.1 0.6 0.3
----------------------------------------------

RMSE: 1.6703
MAE:  1.3894
RMSE: 1.9816
MAE:  1.7140
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.7098
MAE:  1.4213


/data/home/agurov/DeepRecSys/models.py:176: RuntimeWarning: Mean of empty slice
  result = {"rmse":np.sqrt(rmse/cnt), "mae": mae/cnt, "ndcg":np.nanmean(ndcg)}


In [ ]:
from modules import visualize_3d_plot, group_points_by_minimum_error
names = {'SVD': 'svd', 'AutoRec': 'autorec', 'NMF': 'nmf', 'kNN': 'knn'}

In [ ]:
result = result.rename(columns={'model': 'model_name', 'a1': 'a2', 'a2': 'a1'})
result.model_name = result.model_name.apply(lambda x: names[x])
result = result.drop("a3", axis=1)

In [ ]:
rest_sparse = group_points_by_minimum_error(result)

In [ ]:
visualize_3d_plot(rest_sparse, [0.1], metric="RMSE", save_path=None)

In [ ]:
visualize_3d_plot(rest_sparse, [0.5], metric="RMSE", save_path=None)

In [ ]:
visualize_3d_plot(rest_sparse, [0.9], metric="RMSE", save_path=None)